In [5]:
from base_vit import ViT
import torch
from lora import LoRA_ViT
import numpy as np
import matplotlib.pyplot as plt

In [3]:
model = ViT('B_16_imagenet1k')
model.load_state_dict(torch.load('B_16_imagenet1k.pth'))
model.eval()
criterion = torch.nn.CrossEntropyLoss()

In [11]:
# weights = dict(model.named_parameters())
weights = model.state_dict()
trainable_keys = [k for k, v in weights.items() if v.requires_grad]
trainable_weights = {k: v.clone() for k, v in weights.items() if k in trainable_keys}
# print(trainable_weights)
# print(state_dict)

OrderedDict([('class_token', tensor([[[ 5.5241e-03,  1.3605e-02, -2.5943e-01, -1.5176e-02,  4.0256e-01,
           4.6956e-02, -7.6992e-03,  1.1583e-02,  4.1794e-02, -2.2618e-01,
          -8.6113e-03, -1.3967e-02, -1.5123e-02, -1.3010e-02, -2.0492e-02,
          -8.5839e-03,  3.3414e-03,  5.7678e-02,  3.6365e-02, -5.6758e-03,
          -4.7974e-02,  5.9637e-03, -5.0221e-03, -1.3371e-02, -6.9871e-03,
           5.0253e-02,  7.3932e-03, -5.7437e-03,  2.1240e-02, -1.8942e-02,
           1.5181e-03,  2.5193e-02, -2.0468e-02,  1.6701e-02,  1.9216e-02,
          -9.3600e-04,  5.2248e-02,  9.3180e-03, -4.8390e-03,  1.8174e-03,
           1.4953e-02,  1.0364e-02,  1.7181e-02, -2.5523e-03,  6.5110e-02,
           4.3907e-01,  1.6787e-02,  4.5343e-03,  1.9626e-02, -1.3791e-03,
          -5.4258e-03, -4.4043e-02,  3.1062e-03, -1.2638e-02,  8.5818e-03,
          -7.1245e-03,  9.0131e-04, -2.6523e-03,  5.1280e-03, -7.6498e-02,
           6.5924e-04,  2.2964e-03,  7.7230e-03,  1.2007e-02,  4.1369e-

In [ ]:
grid_size = 10
delta = 0.1

loss_grid = np.zeros((grid_size, grid_size))
for i in range(grid_size):
    for j in range(grid_size):
        for k, v in weights.items():
            weights[k] = weights[k] + delta * (i - grid_size // 2) * v + delta * (
                j - grid_size // 2) * torch.randn_like(v)
        model.load_state_dict(weights)
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs, labels

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

        loss_grid[i, j] = running_loss


In [ ]:
plt.imshow(loss_grid, cmap='jet', interpolation='nearest')
plt.colorbar()
plt.show()